In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from tqdm import tqdm
import matplotlib.pyplot as plt

# === 1. Chargement du fichier Excel ===
file_path = r"C:\Users\hp\Desktop\Données Capteurs global with HI.xlsx"
df = pd.read_excel(file_path, sheet_name="Feuil1")
df = df.sort_values("Heure").reset_index(drop=True)

# === 2. Seuils critiques pour chaque capteur ===
seuils = {
    'Temp échappement Droit 540': 618.0,
    'Temp échappement gauche 538': 607.5,
    'Temp liquide refroidissement 525': 102.0,
    'Temp sortie convertisseur 544': 128.5,
    'Pression embrayage impeller 536': 3967.5,
    'Température Essieux avant 531': 69.0,
    'Régime moteur 530 ': 1951.0,  # attention à l'espace
    'Température huile direction 541': 79.5,
    'Température huile freinage 522': 111.0,
    'Température PTO avant 532': 88.5,
    'Pression huile moteur 529': 518.5
}

# === 3. Fonction du processus de Wiener avec Kalman et RUL estimation ===
def wiener_rul_estimation(sensor_series, seuil, mu_init=0.1, P0=1.0, Q=0.01, sigma=1.0):
    X = sensor_series.values
    rul = np.zeros_like(X, dtype=float)

    mu_t = mu_init
    Pt = P0

    for i in range(1, len(X)):
        dt = 1  # pas de temps
        Ft = (dt) * Pt + sigma**2 * dt
        Kt = Pt * dt / Ft
        mu_t = mu_t + Kt * ((X[i] - X[i-1]) - mu_t * dt)
        Pt = Pt - Kt * dt * Pt + Q

        w = seuil
        Xi = X[i]
        if Xi >= w:
            rul[i] = 0
            continue

        # Calcul de la fdp f_t(h_t) (eq 1.27)
        ht_values = np.linspace(1, 200, 200)
        diff = w - Xi
        denom = np.sqrt(2 * np.pi * ht_values**3 * sigma**2)
        exponent = -((diff - mu_t * ht_values)**2) / (2 * ht_values * sigma**2)
        fdp = (diff / denom) * np.exp(exponent)

        integral = np.trapz(fdp, ht_values)
        if integral <= 0 or np.isnan(integral):
            rul[i] = 0
            continue

        fdp /= integral  # normaliser
        rul[i] = np.trapz(ht_values * fdp, ht_values)  # eq 1.28

    return rul

# === 4. Application à tous les capteurs ===
rul_results = {}
for capteur in tqdm(seuils):
    if capteur not in df.columns:
        print(f"Colonne manquante : {capteur}")
        continue
    print(f"Traitement : {capteur}")
    rul_results[capteur] = wiener_rul_estimation(df[capteur], seuils[capteur])

# === 5. Compilation finale ===
rul_df = pd.DataFrame(rul_results)
rul_df["Heure"] = df["Heure"]
rul_df = rul_df.set_index("Heure")

# === 6. Sauvegarde Excel ou CSV ===
rul_df.to_excel("RUL_Wiener_resultats.xlsx")


  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\hp\AppData\Local\Temp\ipykernel_24516\2653494427.py:55: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  integral = np.trapz(fdp, ht_values)
C:\Users\hp\AppData\Local\Temp\ipykernel_24516\2653494427.py:61: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  rul[i] = np.trapz(ht_values * fdp, ht_values)  # eq 1.28


Traitement : Temp échappement Droit 540


  9%|▉         | 1/11 [00:24<04:02, 24.29s/it]

Traitement : Temp échappement gauche 538


 18%|█▊        | 2/11 [00:51<03:53, 25.94s/it]

...

 82%|████████▏ | 9/11 [02:57<00:34, 17.45s/it]

Traitement : Température PTO avant 532


 91%|█████████ | 10/11 [03:11<00:16, 16.27s/it]

Traitement : Pression huile moteur 529


100%|██████████| 11/11 [03:25<00:00, 18.69s/it]


## Precessus de Gamma 

In [3]:
import pandas as pd
import numpy as np
from scipy.special import gammaincc
from tqdm import tqdm

# === 1. Charger les données ===
file_path = r"C:\Users\hp\Desktop\Données Capteurs global with HI.xlsx"
df = pd.read_excel(file_path, sheet_name="Feuil1")
df = df.sort_values("Heure").reset_index(drop=True)

# === 2. Définir les seuils critiques ===
seuils = {
    'Temp échappement Droit 540': 618.0,
    'Temp échappement gauche 538': 607.5,
    'Temp liquide refroidissement 525': 102.0,
    'Temp sortie convertisseur 544': 128.5,
    'Pression embrayage impeller 536': 3967.5,
    'Température Essieux avant 531': 69.0,
    'Régime moteur 530 ': 1951.0,  # attention à l'espace
    'Température huile direction 541': 79.5,
    'Température huile freinage 522': 111.0,
    'Température PTO avant 532': 88.5,
    'Pression huile moteur 529': 518.5
}

# === 3. Fonction d'estimation du RUL selon le processus Gamma (Abdel-Hameed) ===
def gamma_rul_abdel(sensor_series, seuil, u=1.0):
    X = sensor_series.values
    n = len(X)
    RUL = np.zeros(n)
    S = seuil

    for t in range(n):
        xt = X[t]
        if xt >= S:
            RUL[t] = 0
            continue

        v_t = t + 1
        shape = v_t / u
        Su = S * u

        try:
            F_t = gammaincc(shape, Su)
            if F_t <= 0 or np.isnan(F_t):
                RUL[t] = 0
            else:
                Tt = np.mean(np.linspace(t + 1, t + 200, 200))
                RUL[t] = Tt - t
        except:
            RUL[t] = 0

    return RUL

# === 4. Application à tous les capteurs ===
gamma_rul_results = {}

for capteur in tqdm(seuils, desc="Calcul du RUL Gamma"):
    if capteur not in df.columns:
        print(f"⚠️ Colonne introuvable : {capteur}")
        continue
    print(f"→ Traitement : {capteur}")
    gamma_rul_results[capteur] = gamma_rul_abdel(df[capteur], seuils[capteur])

# === 5. Assemblage final ===
gamma_rul_df = pd.DataFrame(gamma_rul_results)
gamma_rul_df["Heure"] = df["Heure"]
gamma_rul_df = gamma_rul_df.set_index("Heure")

# === 6. Sauvegarde Excel ===
gamma_rul_df.to_excel("RUL_Gamma_Resultats.xlsx")
print("✅ Fichier Excel généré : RUL_Gamma_Resultats.xlsx")


Calcul du RUL Gamma:   0%|          | 0/11 [00:00<?, ?it/s]

→ Traitement : Temp échappement Droit 540


Calcul du RUL Gamma:   9%|▉         | 1/11 [00:09<01:31,  9.13s/it]

→ Traitement : Temp échappement gauche 538


Calcul du RUL Gamma:  18%|█▊        | 2/11 [00:17<01:15,  8.39s/it]

...

→ Traitement : Température PTO avant 532


Calcul du RUL Gamma:  91%|█████████ | 10/11 [01:03<00:05,  5.20s/it]

→ Traitement : Pression huile moteur 529


Calcul du RUL Gamma: 100%|██████████| 11/11 [01:07<00:00,  6.14s/it]


✅ Fichier Excel généré : RUL_Gamma_Resultats.xlsx


## Comparaison entre Weiner et Gamma 

In [4]:
# === 8. Analyse comparative pour un capteur ===
capteur_test = "Temp échappement Droit 540"
w = rul_df_wiener[capteur_test]
g = rul_df_gamma[capteur_test]
x = df[capteur_test]
seuil = seuils[capteur_test]

# Métriques
def monotonicity(rul): return np.mean(np.diff(rul) <= 0)
def correlation(rul, x, seuil): return spearmanr(rul, seuil - x)[0]

metrics = pd.DataFrame({
    "Méthode": ["Wiener", "Gamma"],
    "Moyenne RUL": [w.mean(), g.mean()],
    "Écart-type": [w.std(), g.std()],
    "Monotonicité": [monotonicity(w), monotonicity(g)],
    "Corrélation au seuil": [correlation(w, x, seuil), correlation(g, x, seuil)]
})

# === 9. Visualisations ===
plt.figure(figsize=(12, 5))
plt.plot(w, label="Wiener")
plt.plot(g, label="Gamma", alpha=0.7)
plt.title(f"Comparaison RUL estimé - {capteur_test}")
plt.xlabel("Temps (échantillons)")
plt.ylabel("RUL")
plt.legend(); plt.grid(); plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 4))
sns.histplot(w, kde=True, label="Wiener", color="blue")
sns.histplot(g, kde=True, label="Gamma", color="orange", alpha=0.6)
plt.title("Distribution des RUL estimés")
plt.legend(); plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.boxplot(data=[w.dropna(), g.dropna()])
plt.xticks([0, 1], ["Wiener", "Gamma"])
plt.title("Boxplot des RUL"); plt.grid(); plt.tight_layout()
plt.show()

# === 10. Affichage tableau métrique ===
print("\n📋 Comparaison des Méthodes RUL :\n")
print(metrics.round(3))

NameError: name 'rul_df_wiener' is not defined